---
>「成功に説明はいらない。同じように、失敗に言い訳はいらない。」\
>ナポレオン・ヒル
---

# 説明可能AI (Explainable AI)

AIは何が裏で行われているかよくわからないといわれる
- 感覚としては、ここまででDNNの気持ちはわかってもらえていると思う
- ただ、どうしてクラス分けができるのかはともかく、では、そのクラス分けに穴はないのか？例えば、特殊な画像を見せると容易に誤った結果を導いたりしないのか？を示すには証明が必要である
  - 実際にそういった画像は存在する
  - 例えば「かく乱画像」(adversarial image)と呼ばれる改変画像は認識精度を著しく損なうことができる
- ともかくも、AIは判断プロセスがブラックボックスであるとも言われる
- そこで、説明可能AIが提案されている
- GDPR（General Data Protection Regulation = EU一般データ保護規則）では、**消費者の説明を求める権利**が規定されており、このままではAIが利用できません

説明可能AIそのものの定義もかなりあいまい
- 学習過程について、根拠となったデータをきちんと示せば説明可能であるという考え方
  - 方法はブラックボックスだが、このデータのこの部分の特徴量がこの程度結果に関与していると示し、これを人間で確認できれば根拠を示したことになる？
- 解法そのものを証明できる形に変更する
  - 特徴量$x$に対してDNNによる非線形変換$f$があり、結果として$y$を得たとすると、$y=f(x)$という関係式が成り立つ
  - この意味での説明可能性は、$x=f^{-1}(y)$となる$f^{-1}$を導くことであるが、そもそも、次元圧縮(固有値0の固有ベクトルのようなもの)を行うため、逆行列や逆変換は存在しえず、完全に新規の手法

一般に前者の考え方が取られているが、そもそもそれは、**完全な説明可能ではない**
  - 材料を提供するだけで、判断は人間がしなさい、というシステム
  - 説明可能とするために、別のDNNを組み込む提案が殆どで、DNNのブラックボックスは何も説明していない
    - わけのわからないものが出した答えが正しいかわからないので、もう一度わけのわからないものを使って、その原因との関連を見てみましょうという考えはどうなのか？
    - こういう発想は実は多く、インターネットのサイト証明はCAという権威サーバにより証明されたサーバだから信じようという前提で、権威サーバが正しいことは誰も保証していない
    - 安全だとされる内容も、完全は存在せず、ブロックチェーンのビザンチン将軍問題(51%問題)、ハッシュの衝突問題、暗号の確率問題、サイドチャネルアタックなど


## SmoothGrad

わかりやすいメジャーな手法としてSmoothGradがある

その戦略はシンプルで、1枚の入力画像に複数のノイズを加え、それらから生成される複数の反応マップの平均を取ることで注目点を可視化する
- つまり、少しの値の変更が、あるクラスの識別スコアに大きく影響を与えるピクセルがあれば、このピクセルが識別に大きく寄与していると考えている
  - そのようなxの領域を可視化する
  - 当然ながら、ノイズを与えるためその標準偏差と平均化する際のサンプル数といったハイパーパラメータが存在する
  - ノイズは10%から20%程度、サンプル数は多いほどスムーズな結果の画像が得られる

<img src='http://class.west.sd.keio.ac.jp/dataai/text/smoothgrad.jpg' width=600>

## Grad-CAM

GradCAM(Gradient-weighted Class Activation Mapping)は、説明可能AIにおける基本となるアイデアを提供した

CNNを含むニューラルネットワークにおいて画像のどの部分に注目して判断がなされたかを可視化するための技術であり、どの部分に注目したかを示すため、判断材料の根拠の一つを提供する

### 以下の手順で求める

1. 最終出力の結果、maxにより推論の結果として得られたクラスのみを1とし、他を0にした状態で逆誤差伝播を行う
  - 従って、例えば犬と猫が両方映っている場合や、どちらともつきにくい画像である場合も、犬のみを1にする、もしくは猫のみを1にすることで、そのように判断した理由を推し量ることができる
1. 逆伝搬の計算がCNNの最終層に達したら各チャンネル毎に重みをGAP(Global Average Pooling)を用いて計算し、指定されたクラスにおける最終層の各チャンネルの重要度を決定
1. その重要度と各チャンネルの重みを掛けたのち、それらの総和を求める
1. その総和にReLUを施す
  - ReLUを施すのは、クラスに影響を与えてるのは正の勾配を持つ要素に限定されるため、影響を与えない負の値を無視するため
1. 得られた重みに従ってクラスの逆誤差を伝播させて画像を生成する(Guided Backpropagation)

<img src='http://class.west.sd.keio.ac.jp/dataai/text/gradcam.png' width=600>

### 以下の特徴を持つ

- 入力に続いてCNNが存在するネットワーク全てに対応し、CNNの後にどのようなネットワークが繋がっていてもよい(VQA、多クラス分類などでも利用可能)
- 入力画像とその時に推論結果とするクラスの両方の指定が必要
- ネットワークの構造を変えずに可視化できる
- 入力からCNNの層が連続し、その最終層の勾配から可視化する
- 細かい反応部分を求めることができないため、GuidedBackpropと掛け合わせてることで、より詳細な反応部分の可視化ができる
- VQAやResNetの可視化も行えるが、深い層から浅い層に行くにつれて出力サイズが大きく異なるときに精度が悪くなる

そもそも、基本発想は単純かつ、限定的で次の通り
- CNNに続くLinier層(全結合層)は画像の位置に関する情報が連した情報を完全に失ってしまい、また入力に近いところでは抽象度の低い認識しか行われないので、CNNの部分の最終そして全結合層の入力に当たる部分に双方を満たす情報があると考えている。


# GradCAMを試す

In [ ]:
import urllib
import pickle
import cv2
import numpy as np
from PIL import Image

import torch
import torch.nn.functional as F
from torchvision import transforms
from torchvision import models

import matplotlib.pyplot as plt
import seaborn as sns

## ImageNetの読み込み

ImageNetの学習済みモデルを用いる

ラベルとモデルをそれぞれ、labes, modelに読み込む

In [ ]:
labels = pickle.load(urllib.request.urlopen('http://class.west.sd.keio.ac.jp/dataai/data/imagenet1000_clsid_to_human.pkl'))
model = models.vgg19(pretrained=True)
model

## GradCAMの定義

GradCAMでは次の処理を行う
- 畳み込みの最終層でGlobalAveragePoolingを行う
- あるクラスにおける最終層の各チャネルの重要度を決定
- 重要度に応じて各チャンネルをかけて足し合わせる
- 足し合わせたものをReLU関数に通す

GradCamクラスは次の通り

- `__call__`で入力としての画像を与える
  - コードは複数の画像が与えられても動作するように`x.size(0)`の`for`ループがあるが、ここでは一つだけ与えている
  - 受け取った画像を最小を0、最大を1として正規化
  - 特徴量をfeatureに入れる
  - moduleのネットワーク階層を辿る
    - この部分はImageNetに特化している
    - moduleは、features, avgpool, classifierの3つで構成され、これらがこの順にそれぞれ1度だけ選択される
    - classifierになった時にその直前のfeaturesを保持する
    - featuresでは、勾配保存用メソッド`__save_grandient__`を呼び出すことで勾配を保存する
- 各チャネルの重要度を計算してweightへ代入
- 特徴量とweightを掛けて足し合わせる
- 結果のサイズを調整し、正規化、これを元の画像と重ね合わせる


In [ ]:
def z2onorm(d):
  # 単純に最小から最大を0から1で正規化
  d -= np.min(d)
  d /= np.max(d)
  return d

class GradCam:
  def __init__(self, model):
    self.model = model.eval()
    self.feature = None
    self.gradient = None
  def save_gradient(self, grad):
    self.gradient = grad
  def __call__(self, x):
    image_size = (x.size(-1), x.size(-2))
    feature_maps = []
    for i in range(x.size(0)):
      img = z2onorm(x[i].data.numpy()) # 正規化した入力(画像表示のとき元も出すため)
      feature = x[i].unsqueeze(0) # そのままの入力、こちらを利用
      for name, module in self.model.named_children():
        # モデルにあるノード名称を順次取得
        # 実際にはfeatures, avgpool, classifierとなる
        if name == 'classifier': # (4) 最後にclassifier
          print(feature.shape)
          feature = feature.view(feature.size(0), -1)
          # 途中で止めると途中の型変換が行われないのでfeatureの形を変換(表示しているので確認)
          print(feature.shape)
          # classifierであればfeatureを特徴量を取得
        feature = module(feature)
          # ここは3回評価されるが moduleなので、features, avgpool, classifierを順次計算する
          # (1)まずそのままの入力でfeatureだけ計算した出力を得る
          # (3)avgpoolで、featuresの演算を使い続きを計算
          # (4)classifierで、avgpoolの続きをさらに計算
        if name == 'features': # (2-1) 最初はfeature
          feature.register_hook(self.save_gradient)
          # 得た出力で逆伝搬計算する際にフックして勾配を取得するようにする
          self.feature = feature # (2-2) featureを記録
      classes = torch.sigmoid(feature) # sigmoidを求めているが、ここはなくてもよい
      one_hot, _ = classes.max(dim=-1) # ここでワンホット化
      self.model.zero_grad()
      one_hot.backward() # ワンホット化した内容で逆伝搬、上記のフックが働く
      weight = self.gradient.mean(dim=-1, keepdim=True).mean(dim=-2, keepdim=True)
      # フックして取得した勾配から平均を求めることで重要度を計算
      mask = F.relu((weight * self.feature).sum(dim=1)).squeeze(0)
      # 重要度に記録しておいた特徴量を掛け合わせてReLUへ
      mask = cv2.resize(mask.data.cpu().numpy(), image_size)
      mask = z2onorm(mask)
      # 元の絵に重みを重畳して書き込む
      feature_map = np.float32(cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET))
      cam = feature_map + np.float32((np.uint8(img.transpose((1, 2, 0)) * 255)))
      cam = z2onorm(cam)
      feature_maps.append(transforms.ToTensor()(cv2.cvtColor(np.uint8(255 * cam), cv2.COLOR_BGR2RGB)))
          
    feature_maps = torch.stack(feature_maps)
    # torch.Tensorをpyrhon listで結合したので、これをTensorにする
    return feature_maps

In [ ]:
import os
if not os.path.exists('testimg.jpg'):
  #!wget "https://drive.google.com/uc?export=download&id=1nGpIBGphU3uxngGM5kclLj58tFo_ZNf1" -O testimg.jpg
  !wget https://keio.box.com/shared/static/aesvle22z4l8bloc7b494ousjhfbdlt4 -O testimg.jpg

transform = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.ToTensor(),
])

test_image = Image.open("./testimg.jpg")
test_image_tensor = (transform((test_image))).unsqueeze(dim=0)

image_size = test_image.size
print("image size: ", image_size)

plt.imshow(test_image)

In [ ]:
grad_cam = GradCam(model)

feature_image = grad_cam(test_image_tensor).squeeze(dim=0)
feature_image = transforms.ToPILImage()(feature_image)

pred_idx = model(test_image_tensor).max(1)[1]
print("pred: ", labels[int(pred_idx)])
plt.title("Grad-CAM feature image")
plt.imshow(feature_image.resize(image_size))

# 課題

入力画像を変えて、注目点の変化を確認しなさい
